## A Simple [SEIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SEIR_model) Model for Covid-19

#### Author: Dan Segal (@[djsegal](https://github.com/djsegal/))

as requested by Prof. Alan Edelman, MIT

----


In [1]:
using Measurements

param_error = 0.05

α = 0.2 ± param_error
γ = 0.5 ± param_error

# β = f(social distancing, masks, etc.)

E₀ = 1e-4
u₀ = [ 1 - E₀ ; E₀ ; 0 ; 0 ]

tspan = (0.0, 200.0);  # 200 days


In [2]:
using DifferentialEquations

function seir!(du,u,p,t)
  S,E,I,_ = u
  α,β,γ = p

  dS = ( -β*I*S )
  dE = ( +β*I*S ) - α*E
  dI = ( -γ*I   ) + α*E
  dR = ( +γ*I   )

  du .= (dS, dE, dI, dR)
end

labels = ["S", "E", "I", "R"];


In [3]:
using SimplePlots

display(html"""
  <h3>Sliding the β value reduces/increases infectivity</h3>
  <p style="margin-top: 0.4em;">(social distancing, immunity, vaccines reduce β)</p>
""")

@demo for β in slider(0.25:0.25:2, value=1, label="β")  
  title!("Covid-19 SEIR Model with Error Bars") ;
  xlabel!("Time (Days)") ; ylabel!("%")  
  
  soln = solve(ODEProblem(
    seir!, u₀, tspan, [α, β ± param_error, γ]
  ), saveat=0.5)
  
  x, y = soln.t, 100*Array(soln)
  ylims!(0, 100) # using percentages
  
  for (index, label) in enumerate(labels)
    plot!(
      x, y[index,:], label=label, 
      alpha=0.8, fillalpha=0.1
    )
  end
end


HTML{String}("  <script type=\"text/javascript\" class=\"js-plotly-script\">\n    globalComms = {};\n    demoData = {};\n\n    \$(\".js-plotly-script\").parent().css('padding', 0);\n\n    function customPlotlyReact(curPlot, curData, curLayout, curConfig) {\n      for (var i = 0; i < curData.length; i++) {\n        curData[i][\"visible\"] = true;\n        for (var j = 0; j < curPlot.data.length; j++) {\n          if ( curData[i].name !== curPlot.data[j].name ) { continue; }\n          if ( curPlot.data[j].visible !== \"legendonly\" ) { continue; }\n\n          curData[i][\"visible\"] = \"legendonly\";\n        }\n      }\n\n      Plotly.react(curPlot, curData, curLayout, curConfig);\n    }\n\n    function customPlotLoader(curCallback) {\n      if ( \$(\".js-nouislider-css\").length == 0 ) {\n        \$(\"head\").append(\n          '<link class=\"js-nouislider-css\" href=\"https://cdn.jsdelivr.net/gh/djsegal/SimplePlots.jl/dist/nouislider.min.css\" rel=\"stylesheet\">'\n        );\n      }\n\n      if ( \$(\".js-custom-css\").length == 0 ) {\n        \$(\"head\").append(\n          '<link class=\"js-custom-css\" href=\"https://cdn.jsdelivr.net/gh/djsegal/SimplePlots.jl/dist/custom.min.css\" rel=\"stylesheet\">'\n        );\n      }\n\n      if ( typeof Plotly !== \"undefined\" ) {\n        if ( typeof curCallback !== \"undefined\" ) {\n          curCallback();\n        }\n        return;\n      }\n\n      var plotlyScripts = document.getElementsByClassName(\"js-plotly-script\");\n\n      for (var i = 0; i < plotlyScripts.length; i++) {\n        var scriptParent = plotlyScripts[i].parentElement;\n        scriptParent.style.margin = \"0\";\n        scriptParent.style.padding = \"0\";\n      }\n\n      require.config({\n        paths: {\n          Plotly: \"https://cdn.plot.ly/plotly-1.53.0.min\",\n          noUiSlider: \"https://cdn.jsdelivr.net/gh/djsegal/SimplePlots.jl/dist/nouislider.min\",\n          wNumb: \"https://cdn.jsdelivr.net/gh/djsegal/SimplePlots.jl/dist/wnumb.min\"\n        }\n      });\n\n      require([\"Plotly\", \"noUiSlider\", \"wNumb\"], function(Plotly, noUiSlider, wNumb){\n        window.Plotly = Plotly;\n        window.noUiSlider = noUiSlider;\n        window.wNumb = wNumb;\n\n        if ( typeof curCallback !== \"undefined\" ) {\n          curCallback();\n        }\n\n        \$(\"head\").append(\$(\".js-plotly-script\"));\n        \$(\"head .js-plotly-script:not(:first)\").remove();\n        \$(\"body .js-plotly-script\").remove();\n      });\n    }\n\n    \$(document).ready(customPlotLoader);\n    customPlotLoader();\n  </script>\n")

HTML{String}("  <h3>Sliding the β value reduces/increases infectivity</h3>\n  <p style=\"margin-top: 0.4em;\">(social distancing, immunity, vaccines reduce β)</p>\n")

HTML{String}("  <div class='js-interact' id='js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b'>\n    <div class='js-widgets'>\n  <div class='cs-widget js-widget'>\n    <div class='cs-widget-label'>\n      β\n    </div>\n<div class='cs-widget-slider js-widget-slider'></div>  <div class='cs-widget-value js-widget-value '>\n    1.0\n  </div>\n\n  </div>\n    </div>\n\n    <div class='js-display'>\n    </div>\n  </div>\n  <script>\n    var anonFunc = function () {\n  var foundSlider = false;\n  \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .js-widget-slider\").each(function( index ) {\n    var tmpText = \$(this.parentElement).children(\".cs-widget-label\").text().trim();\n    if ( tmpText !== \"β\" ) { return; }\n\n    if ( foundSlider ) { alert(\"There are repeated sliders!\"); }\n\n    foundSlider = true;\n    noUiSlider.create(this, {\n        start: 3,\n        step: 1,\n        connect: \"lower\",\n        format: wNumb({\n          decimals: 0\n        }),\n        range: {\n          min: 0,\n          max: 7\n        }\n    });\n\n    var that = this;\n    this.noUiSlider.on('update', function (values, handle) {\n      var tmpRange = [\"0.25\", \"0.5\", \"0.75\", \"1.0\", \"1.25\", \"1.5\", \"1.75\", \"2.0\"];\n      var newValue = tmpRange[values[handle]];\n\n      var tmpSpan = \$(that.parentElement).children(\".js-widget-value\")[0];\n      tmpSpan.innerText = newValue;\n\n      \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b\").trigger(\"interact\");\n    });\n  });\n  if ( !foundSlider ) { alert(\"Unable to find all sliders!\"); }\n  \$(\".js-widget-toggle .cs-button\").click(function (curEvent) {\n    if ( !\$(curEvent.target).hasClass(\"cs-active\") ) {\n      var curItems = \$(curEvent.target.parentElement).children(\".cs-button\");\n      var curIndex = curItems.index(curEvent.target);\n\n      \$.each(curItems, function(workIndex, workValue) {\n        var otherTarget = \$(curEvent.target.parentElement.children[workIndex]);\n\n        otherTarget.removeClass(\"cs-active\");\n        otherTarget.removeClass(\"cs-pre-active\");\n\n        if ( workIndex == curIndex ) { otherTarget.addClass(\"cs-active\"); }\n        if ( workIndex == curIndex - 1 ) { otherTarget.addClass(\"cs-pre-active\"); }\n      });\n\n      var curWidgetVal = \$(curEvent.target).closest(\".js-widget\").children(\".js-widget-value\")[0];\n      curWidgetVal.innerText = curEvent.target.innerText;\n\n      \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b\").trigger(\"interact\");\n    }\n  });\n  \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b\").on(\"interact\", function() {\n    var msgLabels = \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .cs-widget-label\").map(function(){\n       return \$.trim(\$(this).text());\n    }).get();\n\n    var msgValues = \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .cs-widget-value\").map(function(){\n       return \$.trim(\$(this).text());\n    }).get();\n\n    var msgDict = msgLabels.reduce((obj, k, i) => ({...obj, [k]: msgValues[i] }), {});\n\n    var workPlotlyId = \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .js-plotly-plot\").attr(\"id\");\n    msgDict[\"___interact_plot_id___\"] = workPlotlyId;\n\n    if ( \"interact-d217c8a8-f11f-43ae-97d0-f836b7fdf43b\" in demoData ) {\n      tmpData = demoData[\"interact-d217c8a8-f11f-43ae-97d0-f836b7fdf43b\"];\n  tmpData = tmpData[msgDict[\"β\"]];\n      var workPlot = \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .js-plotly-plot\")[0];\n      var workJson = tmpData;\n\n      if ( typeof workPlot === \"undefined\" ) {\n        var tmpDisplay = \$(\"#js-interact__d217c8a8-f11f-43ae-97d0-f836b7fdf43b .js-display\");\n        var tmpPlotId = \"js-plot-dd16f73a-09e7-4399-9954-0712419a29f5\";\n        tmpDisplay.html('<div id=\"' + tmpPlotId + '\" style=\"width:600px;height:400px;\"></div>');\n\n        plotDiv = document.getElementById(tmpPlotId);\n        Plotly.newPlot(plotDiv, workJson.data, wor